# Analysers in Intel® Sytem Studio

### Profiling a Computer Vision Application

OpenCV in Intel® Distribution of OpenVINO™ leverages a tracing facility for efficient development of Computer Vision Applications.

Tracing framework is built-in directly into the OpenCV source code. To use it, you only need to compile and link your application with modern version of OpenCV.

OpenCV tracing relies on the ITT API, an [Intel®-provided profiling framework](https://software.intel.com/en-us/articles/intel-itt-api-open-source).

In [ ]:
#The tracing is enabled by default at compile time. 
#But it's not activated by default at runtime, because it impacts the performance. 
#It's activated when the environment variable OPENCV_TRACE is set (OPENCV_TRACE=1) 
#or when OpenCV-based app is running within a specialized profiling software supporting 
#Instrumentation and Tracing Technology (ITT).

%env OPENCV_TRACE=0

### Using VTune™ Amplifier

The handiest way to collect a trace is to run your application under control of Intel® VTune™ Amplifier. With VTune™ you can correlate your OpenCV activity with various important metrics for CPU and GPU. Finally, VTune™ captures most Intel APIs beyond OpenCV, for example OpenCL. So with these tools you can see when OpenCV calls are translated into OpenCL tasks.<br><br>
![alt text](VTune1.png) <br>
Just run analyzed OpenCV Application under Intel® VTune™ Amplifier with enabled "Analyze user tasks" option.

In [ ]:
!python rm-rf.py runs

In [ ]:
#VTune Hotspot analysis
!start "" cmd /K amplxe-cl \
-collect hotspots \
--result-dir runs/vtune/cpu_run_1 \
--duration 15 \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"

In [ ]:
#Generate VTune Hotspot analysis report
%sx amplxe-cl \
-report hotspots \
-r=runs/vtune/cpu_run_1 \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"

In [ ]:
#This example displays a Hotspots report for the r001hs result, 
#presenting CPU time for the functions of the target in  
#descending order starting from the most time-consuming function.

%sx amplxe-cl -report hotspots -r runs/vtune/cpu_run_1

In [ ]:
%%capture
#Launch VTune Amplifier 2019 GUI
!"C:\Program Files (x86)\IntelSWTools\VTune Amplifier 2019\bin64\amplxe-gui.exe" runs/vtune/cpu_run_1/cpu_run_1.amplxe

### Using Intel® Advisor™ Amplifier
![alt text](Advise1.png) <br>

<font color=pink><b>Command Line Interface Reference</b></font>

You can use the Intel® Advisor command line interface, advixe-cl, to run analysis and data collection. This makes it possible to automate many tasks or analyze an application running on remote hosts. The results can be viewed through the Intel Advisor GUI, or used to generate various kinds of reports.
<br><br>
NOTE
<br>
Before using the command line, you must set command line environment variables.
For instructions, see [Setting and Using Intel Advisor Environment Variables](https://software.intel.com/en-us/advisor-user-guide-setting-and-using-intel-advisor-environment-variables).

#### Survey Report Purpose and Usage

Run a Survey analysis to generate a <b>Survey Report</b> that offers integrated compiler report data and performance data for your target application all in one place. Optionally run a Trip Counts analysis and/or FLOP analysis to add data to the Survey Report. The Roofline analysis runs a Survey analysis followed by a FLOP analysis automatically.

<font color=blue><b>Survey analysis</b></font> - Identifies:
* Where vectorization, or parallelization with threads, will pay off the most
* If vectorized loops are providing benefit, and if not, why not
* Un-vectorized loops and why they are not vectorized
* Performance problems in general

In [ ]:
#Survey analysis
!start "" cmd /C advixe-cl.exe \
--collect survey \
--duration 15 \
--project-dir runs/advise/cpu_run_1 \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"

In [ ]:
#Create the Survey analysis report
%sx advixe-cl.exe \
--report survey \
--project-dir runs/advise/cpu_run_1 \
--format text \
--report-output=runs/advise/cpu_run_1/cpu_run_1.txt \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"


<font color=blue><b>Trip Counts analysis</b></font> - Dynamically identifies the number of times loops and functions are invoked and executed (also called call count/loop count and iteration count respectively). Use Trip Counts data to:
* Detect loops with too-small trip counts and trip counts that are not a multiple of vector length.
* Analyze parallelism granularity more deeply.

<font color=blue><b>FLOP analysis</b></font> - Dynamically measures floating-point and integer operations, and memory traffic. Use the FLOP analysis to generate application memory usage and performance values that help you make better decisions about your vectorization strategy.

In [ ]:
#Trip counts analysis and FLOP analysis
!start "" cmd /C advixe-cl.exe \
--collect tripcounts -flop -stacks \
--project-dir runs/advise/cpu_run_1 \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"

In [ ]:
!start "" cmd /C advixe-cl.exe \
--collect map –mark-up-list=1,2,3,4 \
--project-dir runs/advise/cpu_run_1 \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"

In [ ]:
!xcopy "C:\Program Files (x86)\IntelSWTools\Advisor\pythonapi\examples" . /s /e

In [ ]:
#Generate a combined report showing all data collected
!"C:\Program Files (x86)\IntelSWTools\Advisor\advixe-vars.bat"
#!python "joined.py" > report.txt

In [ ]:
#Generate an html report
import subprocess
subprocess.call(["advixe-python.exe", "to_html.py runs/advise"])

In [ ]:
!advixe-python.exe to_html.py runs/advise/cpu_run_1

In [ ]:
#You need to do a memory access pattern collection to collect cache statistics
!start "" cmd /C advixe-cl.exe \
--collect map –mark-up-list=4 \
--project-dir advise1 \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"

In [ ]:
#conda install -c blaze sqlite3
!advixe-python.exe cache.py runs/advise/cpu_run_1

In [ ]:
!advixe-python.exe system_info.py runs/advise/cpu_run_1

In [ ]:
%%capture
#Launch Intel Advisor 2019 GUI
!"C:\Program Files (x86)\IntelSWTools\Advisor 2019\bin32\advixe-gui.exe"

# Using Intel® Inspector

Intel® Inspector is a dynamic memory and threading error checking tool for users developing serial and multithreaded applications on Windows* and Linux* operating systems. Follow these steps to get started using the Intel Inspector. <br><br>
![alt text](Inspector1.jpg) <br>

In [ ]:
!start "" cmd /C "C:\Program Files (x86)\IntelSWTools\Inspector 2019\inspxe-vars.bat"
#For the command line interface:
!start "" cmd /C "C:\Program Files (x86)\IntelSWTools\Inspector 2019\bin64\inspxe-cl.exe"

In [ ]:
#List all checking modes
!inspxe-cl.exe -collect-list

```C++
//mm1.cpp

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

void populate(char **str) {
	// 1. Not OK if later freeing the memory (mm1.cpp)
	*str = "Memory";
}

int main() {
	char *s;
	populate(&s);
	printf("%s", s);   // should print "Memory"
	free(s);
	return 0;
}
```

In [ ]:
%sx inspxe-cl -collect mi1 -r runs/inspect/mi1 mm1.exe

In [ ]:
%sx inspxe-cl -create-suppression-file "runs/inspect/mi1/suppressions/mySup" -r runs/inspect/mi1

```C++
//mm2.cpp

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

void populate(char **str) {
	// 1. OK (mm1.cpp)
	*str = (char *)malloc(sizeof(char) * 7);
	strcpy(*str, "Memory");
}

int main() {
	char *s;
	populate(&s);
	printf("%s", s);   // should print "Memory"
	free(s);
	return 0;
}

```

In [ ]:
%sx inspxe-cl -collect mi1 -r runs/inspect/mi12 mm2.exe

```C++
// lockGuardDeadlock.cpp

#include <iostream>
#include <chrono>
#include <mutex>
#include <thread>


int main() {
    std::mutex m1;
    std::mutex m2;
    std::thread t1([&m1, &m2] {
        std::cout<<"\n1. Acquiring m1.\n";
        m1.lock();
        std::this_thread::sleep_for(std::chrono::milliseconds(10));
        std::cout<<"\n1. Acquiring m2\n";
        m2.lock(); 
    });
    std::thread t2([&m1, &m2] {
        std::cout<<"\n2. Acquiring m2\n";
        m2.lock();
        std::this_thread::sleep_for(std::chrono::milliseconds(10));
        std::cout<<"\n2. Acquiring m1\n";
        m1.lock();
    });

    t1.join();
    t2.join();
	std::cout<<std::endl;
}

```

In [ ]:
!start "" cmd /k inspxe-cl -collect ti2 -r runs/inspect/ti2 deadlock.exe

In [ ]:
#create supression file
%sx inspxe-cl.exe \
-r "runs/inspect/ti2" \
-create-suppression-file "runs\inspect\ti2\suppressions\mysup"

In [ ]:
#Reporting: regression status
!inspxe-cl -report status –r runs/inspect/ti2

In [ ]:
#Report the observations
%sx inspxe-cl -report observations  -r runs/inspect/ti2/

In [ ]:
#Reporting: problem list
!inspxe-cl -report problems –r runs/inspect/ti2

In [ ]:
#Reporting: extended problem list
!inspxe-cl -report problems -report-all –r runs/inspect/ti2

In [ ]:
#Reporting: other CLI options
#XML output
!inspxe-cl -report problems –format=xml -r runs/inspect/ti2

In [ ]:
#CSV output
!inspxe-cl -report problems -format csv -csv-delimiter " " -report-output runs/inspect/ti2/observations.csv -r runs/inspect/ti2

In [ ]:
import csv
import sys
with open("runs/inspect/ti2/observations.csv", 'r') as csvfile:
		reader = csv.reader(csvfile)
	 
		for row in reader:
			print(row)

In [ ]:
#Export full result
!inspxe-cl -export -archive-name runs/inspect/ti2/ti2.inspxez -include-sources -result-dir runs/inspect/ti2

In [ ]:
%%capture
#Launch Intel Inspector 2019 GUI
!"C:\Program Files (x86)\IntelSWTools\Inspector 2019\bin64\inspxe-gui.exe"

In [ ]:
#At the end, set the OpenCV tracing flag to zero.
%env OPENCV_TRACE=0